<a href="https://colab.research.google.com/github/KorStats/skku/blob/main/code/EDA%20%EA%B3%BC%EC%A0%95%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 27.9 MB/s eta 0:00:00


In [16]:
import pickle
import random
from konlpy.tag import Okt

# 동의어 사전 로드
def load_synonym_dict(pickle_file):
    with open(pickle_file, 'rb') as f:
        synonym_dict = pickle.load(f)
    return synonym_dict

# Random Swap (랜덤 단어 교환)
def random_swap(words, n=1):
    words = words.copy()
    for _ in range(n):
        idx1, idx2 = random.sample(range(len(words)), 2)
        words[idx1], words[idx2] = words[idx2], words[idx1]
    return words

# Random Insertion (랜덤 단어 삽입)
def random_insertion(words, synonym_dict, n=1):
    words = words.copy()
    for _ in range(n):
        idx = random.randint(0, len(words) - 1)
        word = words[idx]
        if word in synonym_dict:
            words.insert(idx, random.choice(synonym_dict[word]))
    return words

# Random Delete (랜덤 단어 삭제)
def random_delete(words, p=0.1):
    if len(words) == 1:
        return words
    return [word for word in words if random.random() > p]

# Synonym Replacement (동의어 대체)
def synonym_replacement(words, synonym_dict, n=1):
    words = words.copy()
    for _ in range(n):
        idx = random.randint(0, len(words) - 1)
        word = words[idx]
        if word in synonym_dict:
            words[idx] = random.choice(synonym_dict[word])
    return words

# 데이터 증강 실행 함수
#def augment_text(text, synonym_dict, swap_n=1, insert_n=1, delete_p=0.1, replace_n=1):
#    okt = Okt()
#    words = okt.morphs(text)  # Okt를 이용한 토큰화
#    words = random_swap(words, swap_n)
#    words = random_insertion(words, synonym_dict, insert_n)
#    words = random_delete(words, delete_p)
#    words = synonym_replacement(words, synonym_dict, replace_n)
#    return ' '.join(words)

In [21]:
# 데이터 증강 실행 함수
def augment_text(text, synonym_dict, swap_n=0, insert_n=0, delete_p=0, replace_n=0):
    okt = Okt()
    words = okt.morphs(text)  # Okt를 이용한 토큰화

    if swap_n > 0:
        words = random_swap(words, swap_n)
    if insert_n > 0:
        words = random_insertion(words, synonym_dict, insert_n)
    if delete_p > 0:
        words = random_delete(words, delete_p)
    if replace_n > 0:
        words = synonym_replacement(words, synonym_dict, replace_n)

    return ' '.join(words)

In [22]:
# 동의어 사전 로드
synonym_dict = load_synonym_dict('/content/wordnet.pickle')

# 테스트 세부 과정

In [23]:
# 테스트 문장 및 개별 증강 방식 적용
text1 = "보안 분석가는 네트워크 취약점을 식별하고 보호 조치를 수행합니다."
text2 = "기업의 정보 보안 정책을 강화하기 위한 절차를 수립합니다."
text3 = "데이터 보호를 위해 최신 암호화 기술을 적용합니다."
text4 = "침해 사고 대응 프로세스를 구축하고 운영합니다."

aug_swap = augment_text(text1, synonym_dict, swap_n=2)
aug_insert = augment_text(text2, synonym_dict, insert_n=2)
aug_delete = augment_text(text3, synonym_dict, delete_p=0.2)
aug_replace = augment_text(text4, synonym_dict, replace_n=2)

print("원본 문장 1:", text1)
print("Swap 적용:", aug_swap)
print("\n원본 문장 2:", text2)
print("Insertion 적용:", aug_insert)
print("\n원본 문장 3:", text3)
print("Delete 적용:", aug_delete)
print("\n원본 문장 4:", text4)
print("Replace 적용:", aug_replace)


원본 문장 1: 보안 분석가는 네트워크 취약점을 식별하고 보호 조치를 수행합니다.
Swap 적용: 네트워크 분석 가는 보안 취약점 을 보호 하고 식별 조치 를 수행 합니다 .

원본 문장 2: 기업의 정보 보안 정책을 강화하기 위한 절차를 수립합니다.
Insertion 적용: 기업 의 정보 보안 정책 을 강화하기 위 한 절차 를 수립 합니다 .

원본 문장 3: 데이터 보호를 위해 최신 암호화 기술을 적용합니다.
Delete 적용: 를 위해 암호 을 적용 합니다 .

원본 문장 4: 침해 사고 대응 프로세스를 구축하고 운영합니다.
Replace 적용: 침해 사고 대응 프로세스 를 구축 하고 운영 합니다 .
